In [1]:
import os
import sys
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import numpy as np
import datetime
import time
import dask.dataframe as dd
import seedir as sd

import warnings
warnings.filterwarnings("ignore")

## Loading

The data was loaded and transofrmed via connecting to the database with Spark SQL  
Naturaly this data must not be extracted to the public repos so for the purpose of demonstration I have made up a few cedits to show typical cases  

## Detailed approach

In [2]:
sd.seedir(style='emoji', itemlimit=10, beyond='ellipsis', exclude_folders=['.git', '.ipynb_checkpoints'])

📁 InterestCharging/
├─📁 data/
│ ├─📄 ActualOperations.csv
│ └─📄 Rates.csv
├─📄 InterestCharging.ipynb
└─📁 output/
  ├─📄 df_results_detailed.csv
  └─📄 df_results_expedited.csv


In [3]:
# DataFrame containing interest rates of the credits

df_rates = pd.read_csv('data/Rates.csv')
df_rates['c_rate_begin_date'] = pd.to_datetime(df_rates['c_rate_begin_date'])
df_rates['c_rate_end_date'] = pd.to_datetime(df_rates['c_rate_end_date'])
df_rates

,credit_id,c_rate_begin_date,c_rate_end_date,c_rate
0,1,2021-01-01,NaT,11.90
1,2,2021-01-01,2021-08-26,16.90
2,2,2021-08-27,2022-02-01,1.69
3,2,2022-02-02,NaT,16.90
4,3,2021-01-01,NaT,14.95
5,4,2021-01-01,NaT,15.90


In [4]:
# DataFrame containing actual operations of the credits

df_ao = pd.read_csv('data/ActualOperations.csv')
df_ao['c_date'] = pd.to_datetime(df_ao['c_date'])
df_ao

,credit_id,c_date,c_oper_name,c_amount
0,1,2021-01-01,Credit issuing,341582.70
1,1,2021-01-03,Charging interest,222.73
2,1,2021-01-03,Credit repayment,1884.43
3,1,2021-01-03,Credit repayment,1426.00
4,1,2021-01-09,Charging interest,661.72
...,...,...,...,...
281,4,2022-02-15,Charging interest (on debt),904.05
282,4,2022-02-28,Charging interest (on debt),790.60
283,4,2022-03-15,Charging interest (on debt),944.97
284,4,2022-03-31,Charging interest (on debt),1016.70


In [5]:
class Credit():
    '''
        Class contains all the information about the credit 
         and allows to recalculate the amount of the following operations:
         'Charging interest', 'Charging interest (on debt)', 'Charging interest on the date of restructurization'
    '''
    
    def __init__(self, df_rates, df_ao):
        
        self.credit_ID = df_rates.iloc[0]['credit_id']
        self.interest_rate = df_rates[['c_rate', 'c_rate_begin_date']]
        self.df_ao = df_ao
        self.df_ao['c_date'] = pd.to_datetime(self.df_ao['c_date'])
        self.opers_to_observe = ['Charging interest', 'Charging interest (on debt)', 'Charging interest on the date of restructurization']
        if len(df_ao):
            self.amount = df_ao[df_ao['c_oper_name'] == 'Credit issuing'].iloc[0]['c_amount']
            self.issue_date = df_ao[df_ao['c_oper_name'] == 'Credit issuing']['c_date'].iloc[0]
            self.last_date = df_ao['c_date'].iloc[-1]


        
        
                
    def set_per_day_info(self):
        '''
            Creates a 'df_daily' DataFrame with the following columns:
                date: contains all dates between the issue date and the last operation date
                interest_rate: contains the interest rates which were valid at each date from 'date' column
                days_in_year: contains the number of days in the year of each date from 'date' column
                balance_owed: contains the amount of balance owed at each date from 'date' column
                debt_owed: contains the amount of debt owed at each date from 'date' column
        '''
        
        df_daily = pd.DataFrame()
        df_daily['date'] = pd.date_range(self.issue_date, self.last_date, freq='D')
        df_daily['interest_rate'] = df_daily['date'].apply(lambda date: self.interest_rate[(date >= self.interest_rate['c_rate_begin_date'])]\
                                                     ['c_rate'].iloc[-1])
        df_daily['days_in_year'] = df_daily['date'].apply(lambda date: 366 if (date.year % 4 == 0) and (date.year % 100 != 0) or (date.year % 400 == 0) else 365)
        df_daily['balance_owed'] = df_daily['date'].apply(lambda date: self.amount - \
                                                pd.to_numeric(self.df_ao[(self.df_ao['c_date'] <= date) \
                                                                       & (self.df_ao['c_oper_name'].isin(['Credit repayment']))]['c_amount']).sum() - \
                                                pd.to_numeric(self.df_ao[(self.df_ao['c_date'] <= date) \
                                                                       & (self.df_ao['c_oper_name'].isin(['Overdue payment']))]['c_amount']).sum())
        df_daily['debt_owed'] = df_daily['date'].apply(lambda date: \
                                                pd.to_numeric(self.df_ao[(self.df_ao['c_date'] <= date) \
                                                                       & (self.df_ao['c_oper_name'].isin(['Overdue payment']))]['c_amount']).sum() - \
                                                pd.to_numeric(self.df_ao[(self.df_ao['c_date'] <= date) \
                                                                       & (self.df_ao['c_oper_name'].isin(['Debt repayment']))]['c_amount']).sum())
        self.df_daily = df_daily
        
        

                    
    def set_df_interest(self):
        '''
            Creates a 'df_interest' DataFrame with the following columns:
                credit_id: a slice of the df_ao containing operations of charging interest
                c_date: a slice of the df_ao containing operations of charging interest
                c_oper_name: a slice of the df_ao containing operations of charging interest
                c_amount_fact: a slice of the df_ao containing operations of charging interest ('c_amount' column)
                c_amount_exp: the expected amount of the 'charging interest' operations calculated via applying 'get_interest'
                c_amount_exp - c_amount_fact: the difference of the according columns
        '''
        
        df_interest = pd.DataFrame()
        
        if len(self.df_ao) == 0:
            self.diff = np.NaN
            self.abs_diff = np.NaN
            self.df_interest = df_interest
        else:
            df_interest = self.df_ao[self.df_ao['c_oper_name'].isin(self.opers_to_observe)].rename({'c_amount': 'c_amount_fact'}, axis='columns')
            df_interest['c_date'] = pd.to_datetime(df_interest['c_date'])
            df_interest['credit_id'] = self.credit_ID

            self.set_per_day_info()

            def get_interest(row):
                '''
                    Returns the amount of interest charged on a particular operation
                    The sum is calculated via multiplying corresponding columns of 'df_daily' slice* and summing the resulting productions
                    * the slice is made via picking only those dates which account for the charging period
                '''
                
                oper_name = row['c_oper_name']
                date = row['c_date']

                if oper_name in ['Charging interest', 'Charging interest on the date of restructurization']:
                    prev_date = self.df_ao[(self.df_ao['c_date'] < date) & (self.df_ao['c_oper_name'].isin(\
                                           ['Credit issuing', 'Charging interest', 'Charging interest on the date of restructurization']))]\
                                          ['c_date'].iloc[-1]
                    df_daily_i = self.df_daily[(self.df_daily['date'] >= prev_date) & (self.df_daily['date'] < date)]
                    return (df_daily_i['balance_owed'] * df_daily_i['interest_rate'] / 100 / df_daily_i['days_in_year']).sum()
                
                else: # if oper_name == 'Charging interest (on debt)':
                    prev_date = self.df_ao[(self.df_ao['c_date'] < date)][self.df_ao['c_oper_name'].isin(\
                                           ['Overdue payment', 'Charging interest (on debt)'])]\
                                          ['c_date'].iloc[-1]
                    df_daily_i = self.df_daily[(self.df_daily['date'] >= prev_date) & (self.df_daily['date'] < date)]
                    return (df_daily_i['debt_owed'] * df_daily_i['interest_rate'] / 100 / df_daily_i['days_in_year']).sum()


                

            df_interest['c_amount_exp'] = df_interest.apply(lambda row: get_interest(row), axis=1)
            df_interest['c_amount_exp - c_amount_fact'] = df_interest['c_amount_exp'] - df_interest['c_amount_fact']

            self.diff = df_interest['c_amount_exp - c_amount_fact'].sum()
            self.abs_diff = df_interest['c_amount_exp - c_amount_fact'].apply(lambda x: abs(x)).sum()
            self.df_interest = df_interest

In [6]:
list_of_credit_ids = list(df_rates['credit_id'].drop_duplicates())
start_time = time.time()

credits = []

# resulting DataFrame
df_results = pd.DataFrame(columns=['credit ID', 'difference', 'abs difference'])

for i, credit_id in enumerate(list_of_credit_ids):
    
    # slices of df_rates and df_ao which relate to the particular credit to feed into Credit constructor
    df_rates_i = df_rates[df_rates['credit_id'] == credit_id].reset_index(drop=True)
    df_ao_i = df_ao[df_ao['credit_id'] == credit_id].reset_index(drop=True)

    credit_i = Credit(df_rates_i, df_ao_i)
    credit_i.set_df_interest()
    df_results.loc[len(df_results)] = [credit_id, credit_i.diff, credit_i.abs_diff]
    
    credits.append(credit_i)
    
    df_results.to_csv('output/df_results_detailed.csv', index=None)
    
    print(f'Credit {i} is processed; time taken = {round(time.time() - start_time, 2)}')

Credit 0 is processed; time taken = 2.26
Credit 1 is processed; time taken = 4.48
Credit 2 is processed; time taken = 6.62
Credit 3 is processed; time taken = 8.82


In [7]:
df_results['credit ID'] = df_results['credit ID'].astype(int)
df_results

,credit ID,difference,abs difference
0,1,-0.03,0.03
1,2,-0.02,0.02
2,3,-2.80,430.56
3,4,-2430.74,2430.74


In [8]:
pd.options.display.max_rows = 320
def get_df_ao_with_pred(credit_id):
    '''
        Returns a DataFrame with actual operations and expected amounts for 'charging interest' operations for <credit_id>
    '''
    i = df_results[df_results['credit ID'] == credit_id].index[0]
    return pd.merge(credits[i].df_ao, credits[i].df_interest, how='left', on=['credit_id', 'c_date', 'c_oper_name'])\
            .drop('c_amount_fact', axis='columns').rename({'c_amount': 'c_amount_fact'}, axis='columns')

In [9]:
# everything seems fine here

get_df_ao_with_pred(1)

,credit_id,c_date,c_oper_name,c_amount_fact,c_amount_exp,c_amount_exp - c_amount_fact
0,1,2021-01-01,Credit issuing,341582.70,nan,nan
1,1,2021-01-03,Charging interest,222.73,222.73,-0.00
2,1,2021-01-03,Credit repayment,1884.43,nan,nan
3,1,2021-01-03,Credit repayment,1426.00,nan,nan
4,1,2021-01-09,Charging interest,661.72,661.72,-0.00
5,1,2021-01-09,Credit repayment,500.84,nan,nan
6,1,2021-01-24,Charging interest,1651.84,1651.84,-0.00
7,1,2021-01-24,Credit repayment,1779.96,nan,nan
8,1,2021-01-29,Charging interest,547.71,547.71,-0.00
9,1,2021-01-29,Credit repayment,596.71,nan,nan


In [10]:
# and here

get_df_ao_with_pred(2)

,credit_id,c_date,c_oper_name,c_amount_fact,c_amount_exp,c_amount_exp - c_amount_fact
0,2,2021-01-01,Credit issuing,110434.20,nan,nan
1,2,2021-01-07,Charging interest,306.80,306.80,-0.00
2,2,2021-01-07,Credit repayment,1863.95,nan,nan
3,2,2021-01-30,Charging interest,1156.20,1156.20,-0.00
4,2,2021-01-30,Credit repayment,116.21,nan,nan
5,2,2021-01-31,Charging interest,50.22,50.22,-0.00
6,2,2021-02-01,Charging interest,50.22,50.22,-0.00
7,2,2021-02-06,Charging interest,251.08,251.08,-0.00
8,2,2021-02-06,Credit repayment,808.16,nan,nan
9,2,2021-02-27,Charging interest,1046.67,1046.67,-0.00


In [11]:
# the problem seems to be at 2021-04-05 and 2021-05-05

get_df_ao_with_pred(3)

,credit_id,c_date,c_oper_name,c_amount_fact,c_amount_exp,c_amount_exp - c_amount_fact
0,3,2021-01-01,Credit issuing,542984.40,nan,nan
1,3,2021-01-05,Charging interest,889.60,889.60,-0.00
2,3,2021-01-31,Charging interest,5782.41,5782.41,-0.00
3,3,2021-02-05,Charging interest,1112.00,1112.00,-0.00
4,3,2021-02-05,Credit repayment,6667.96,nan,nan
5,3,2021-02-28,Charging interest,5052.40,5052.39,-0.00
6,3,2021-03-05,Charging interest,1098.35,1098.35,-0.00
7,3,2021-03-05,Credit repayment,7337.93,nan,nan
8,3,2021-03-31,Charging interest,5633.26,5633.26,-0.00
9,3,2021-04-05,Charging interest,1299.98,1083.32,-216.66


In [12]:
# let us have a look at daily info at these periods

df_daily = credits[2].df_daily
df_daily[(df_daily['date'] >= datetime.datetime(2021, 4, 1)) & (df_daily['date'] <= datetime.datetime(2021, 4, 10))]

,date,interest_rate,days_in_year,balance_owed,debt_owed
90,2021-04-01,14.95,365,528978.51,0.00
91,2021-04-02,14.95,365,528978.51,0.00
92,2021-04-03,14.95,365,528978.51,0.00
93,2021-04-04,14.95,365,528978.51,0.00
94,2021-04-05,14.95,365,522156.74,0.00
95,2021-04-06,14.95,365,522156.74,0.00
96,2021-04-07,14.95,365,522156.74,0.00
97,2021-04-08,14.95,365,522156.74,0.00
98,2021-04-09,14.95,365,522156.74,0.00
99,2021-04-10,14.95,365,522156.74,0.00


In [13]:
# the charging period for 2021-04-05 is [2021-03-31, 2021-04-05)

print(f'The daily interest for this period is {round(528978.51 * 0.1495 / 365,  2)}')

# which is exactly the amount of mistake

The daily interest for this period is 216.66


In [14]:
# let us have a look at daily info at these periods

df_daily = credits[2].df_daily
df_daily[(df_daily['date'] >= datetime.datetime(2021, 5, 1)) & (df_daily['date'] <= datetime.datetime(2021, 5, 10))]

,date,interest_rate,days_in_year,balance_owed,debt_owed
120,2021-05-01,14.95,365,522156.74,0.00
121,2021-05-02,14.95,365,522156.74,0.00
122,2021-05-03,14.95,365,522156.74,0.00
123,2021-05-04,14.95,365,522156.74,0.00
124,2021-05-05,14.95,365,515066.16,0.00
125,2021-05-06,14.95,365,515066.16,0.00
126,2021-05-07,14.95,365,515066.16,0.00
127,2021-05-08,14.95,365,515066.16,0.00
128,2021-05-09,14.95,365,515066.16,0.00
129,2021-05-10,14.95,365,515066.16,0.00


In [15]:
# the charging period for 2021-05-05 is [2021-04-30, 2021-05-05)

print(f'The daily interest for this period is {round(522156.74 * 0.1495 / 365,  2)}')

# again - it is exactly the amount of mistake

The daily interest for this period is 213.87


In [16]:
# the deviations appear after 2021-09-30

get_df_ao_with_pred(4)

,credit_id,c_date,c_oper_name,c_amount_fact,c_amount_exp,c_amount_exp - c_amount_fact
0,4,2021-01-01,Credit issuing,105485.23,nan,nan
1,4,2021-01-15,Charging interest,643.32,643.32,-0.00
2,4,2021-01-31,Charging interest,735.22,735.22,-0.00
3,4,2021-02-15,Charging interest,689.27,689.27,-0.00
4,4,2021-02-15,Overdue payment,3359.26,nan,nan
5,4,2021-02-28,Charging interest,578.34,578.34,-0.00
6,4,2021-02-28,Charging interest (on debt),19.02,19.02,-0.00
7,4,2021-03-15,Charging interest,667.32,667.32,-0.00
8,4,2021-03-15,Overdue payment,3549.33,nan,nan
9,4,2021-03-15,Charging interest (on debt),21.95,21.95,-0.00


In [17]:
# the problem with this one appeared to be due to charging penalty interest to this credit (as there were no credit repayments)
# however those penalties should have been made as a different operation
# as for this credit, there were by mistake added to usual charging operations

## Expedited option

In [18]:
pd.options.display.max_rows = 60
sd.seedir(style='emoji', itemlimit=10, beyond='ellipsis', exclude_folders=['.git', '.ipynb_checkpoints'])

📁 InterestCharging/
├─📁 data/
│ ├─📄 ActualOperations.csv
│ └─📄 Rates.csv
├─📄 InterestCharging.ipynb
└─📁 output/
  ├─📄 df_results_detailed.csv
  └─📄 df_results_expedited.csv


In [19]:
# DataFrame containing interest rates of the credits

df_rates = pd.read_csv('data/Rates.csv')
df_rates['c_rate_begin_date'] = pd.to_datetime(df_rates['c_rate_begin_date'])
df_rates['c_rate_end_date'] = pd.to_datetime(df_rates['c_rate_end_date'])
df_rates

,credit_id,c_rate_begin_date,c_rate_end_date,c_rate
0,1,2021-01-01,NaT,11.90
1,2,2021-01-01,2021-08-26,16.90
2,2,2021-08-27,2022-02-01,1.69
3,2,2022-02-02,NaT,16.90
4,3,2021-01-01,NaT,14.95
5,4,2021-01-01,NaT,15.90


In [20]:
# DataFrame containing actual operations of the credits

df_ao = pd.read_csv('data/ActualOperations.csv')
df_ao['c_date'] = pd.to_datetime(df_ao['c_date'])
df_ao

,credit_id,c_date,c_oper_name,c_amount
0,1,2021-01-01,Credit issuing,341582.70
1,1,2021-01-03,Charging interest,222.73
2,1,2021-01-03,Credit repayment,1884.43
3,1,2021-01-03,Credit repayment,1426.00
4,1,2021-01-09,Charging interest,661.72
...,...,...,...,...
281,4,2022-02-15,Charging interest (on debt),904.05
282,4,2022-02-28,Charging interest (on debt),790.60
283,4,2022-03-15,Charging interest (on debt),944.97
284,4,2022-03-31,Charging interest (on debt),1016.70


In [21]:
# encode the operations

df_oper_codes = {'Credit issuing': 0,
                 'Charging interest': 1,
                 'Overdue payment': 2,
                 'Charging interest (on debt)': 3,
                 'Credit repayment': 4,
                 'Debt repayment': 5,
                 'Charging interest on the date of restructurization': 6,
                 'Changing interest rate': 7}

In [22]:
# consider the establishing of the interest rate as an actual operation to concat both DataFrames into one

df_rates_m = df_rates.rename({'c_rate_begin_date': 'c_date', 'c_rate': 'c_amount'}, axis='columns')\
                     .drop('c_rate_end_date', axis='columns')
df_rates_m['c_oper_name'] = 'Changing interest rate'
df_rates_m = df_rates_m[['credit_id', 'c_date', 'c_oper_name', 'c_amount']]
df_rates_m

,credit_id,c_date,c_oper_name,c_amount
0,1,2021-01-01,Changing interest rate,11.90
1,2,2021-01-01,Changing interest rate,16.90
2,2,2021-08-27,Changing interest rate,1.69
3,2,2022-02-02,Changing interest rate,16.90
4,3,2021-01-01,Changing interest rate,14.95
5,4,2021-01-01,Changing interest rate,15.90


In [23]:
# concat and sort the united DataFrame
df_opers = pd.concat([df_ao, df_rates_m])
df_opers['c_oper_name'] = df_opers['c_oper_name'].map(df_oper_codes)
df_opers = df_opers.sort_values(['credit_id', 'c_date', 'c_oper_name'])

# add a fake credit in the end of the DataFrame 
# (because the sum of deviations is calculated and saved when we see a new 'Credit Issuing' operation)
df_opers.loc[len(df_opers)] = [-1, datetime.datetime(2100, 1, 1), 0, -1.]

# map all the credit ids in order for them to occupy as little memory space as possible
mapping = {item: i for i, item in enumerate(df_opers['credit_id'].unique())}
df_opers['credit_id'] = df_opers['credit_id'].apply(lambda x: mapping[x])

df_opers = df_opers.reset_index(drop=True)
df_opers

,credit_id,c_date,c_oper_name,c_amount
0,0,2021-01-01,0,341582.70
1,0,2021-01-01,7,11.90
2,0,2021-01-03,1,222.73
3,0,2021-01-03,4,1884.43
4,0,2021-01-03,4,1426.00
...,...,...,...,...
288,3,2022-02-28,3,790.60
289,3,2022-03-15,3,944.97
290,3,2022-03-31,3,1016.70
291,3,2022-04-15,3,991.34


In [24]:
df_opers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293 entries, 0 to 292
Data columns (total 4 columns):
credit_id      293 non-null int64
c_date         293 non-null datetime64[ns]
c_oper_name    293 non-null int64
c_amount       293 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 9.3 KB


In [25]:
# downcast all integer and float columns to reduce memory usage

fcols = df_opers.select_dtypes('float').columns
icols = df_opers.select_dtypes('integer').columns

df_opers[fcols] = df_opers[fcols].apply(pd.to_numeric, downcast='float')
df_opers[icols] = df_opers[icols].apply(pd.to_numeric, downcast='integer')

df_opers.dtypes

credit_id                int8
c_date         datetime64[ns]
c_oper_name              int8
c_amount              float32
dtype: object

In [26]:
df_opers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293 entries, 0 to 292
Data columns (total 4 columns):
credit_id      293 non-null int8
c_date         293 non-null datetime64[ns]
c_oper_name    293 non-null int8
c_amount       293 non-null float32
dtypes: datetime64[ns](1), float32(1), int8(2)
memory usage: 4.1 KB


In [27]:
# create Dask DataFrame from df_opers
ddf_opers = dd.from_pandas(df_opers, npartitions=1)

In [28]:
start_time = time.time()

current_id = 0
current_balance_owed = 0
current_debt_owed = 0
current_balance_previous_date = datetime.datetime(2000, 1, 1)
current_debt_previous_date = datetime.datetime(2000, 1, 1)
current_rate = 0
current_diff = 0
current_abs_diff = 0
current_balance_payment = 0
current_debt_payment = 0


df_results = pd.DataFrame(columns=['credit ID', 'difference', 'abs difference'])

for op in ddf_opers.itertuples():
    if op[3] == 0:                       # Credit issuing
        df_results.loc[len(df_results)] = [current_id, current_diff, current_abs_diff]
        current_diff = 0
        current_abs_diff = 0
        current_id = op[1]
        current_balance_owed = op[4]
        current_debt_owed = 0
        current_balance_payment = 0
        current_debt_payment = 0
        current_rate = 0
        current_balance_previous_date = op[2]
        
    elif op[3] in [1, 6]:                # Charging interest / Charging interest on the date of restructurization
        current_balance_payment += current_balance_owed * (op[2] - current_balance_previous_date).days * current_rate / 100 / 365
        current_diff += (current_balance_payment - op[4])
        current_abs_diff += abs(current_balance_payment - op[4])
        current_balance_previous_date = op[2]
        # df_opers.loc[op[0], 'exp'] = current_balance_payment # logging
        current_balance_payment = 0
        
    elif op[3] == 2:                     # Overdue payment
        if current_debt_owed != 0:
            current_debt_payment += current_debt_owed * (op[2] - current_debt_previous_date).days * current_rate / 100 / 365
        current_debt_owed += op[4]
        current_balance_owed -= op[4]
        current_debt_previous_date = op[2]
        
    elif op[3] == 3:                     # Charging interest (on debt)
        current_debt_payment += current_debt_owed * (op[2] - current_debt_previous_date).days * current_rate / 100 / 365
        current_diff += (current_debt_payment - op[4])
        current_abs_diff += abs(current_debt_payment - op[4])
        current_debt_previous_date = op[2]
        # df_opers.loc[op[0], 'exp'] = current_debt_payment # logging
        current_debt_payment = 0
        
    elif op[3] == 4:                     # Credit repayment
        current_balance_owed -= op[4]
        
    elif op[3] == 5:                     # Debt repayment
        current_debt_payment += current_debt_owed * (op[2] - current_debt_previous_date).days * current_rate / 100 / 365
        current_debt_owed -= op[4]
        current_debt_previous_date = op[2]

    elif op[3] == 7:                     # Changing interest rate  
        current_balance_payment += current_balance_owed * (op[2] - current_balance_previous_date).days * current_rate / 100 / 365
        current_balance_previous_date = op[2]
        current_debt_payment += current_debt_owed * (op[2] - current_debt_previous_date).days * current_rate / 100 / 365
        current_debt_previous_date = op[2]
        current_rate = op[4]
    
    df_results.to_csv('output/df_results_expedited.csv')
    
print(f'All credits are processed; time taken = {round(time.time() - start_time, 2)}')

All credits are processed; time taken = 0.47


In [29]:
df_results['credit ID'] = df_results['credit ID'].astype(int)
# dropping the 'fake' credit
df_results = df_results.drop(0)
df_results

,credit ID,difference,abs difference
1,0,-0.03,0.03
2,1,-0.02,0.02
3,2,-2.81,430.57
4,3,-2430.74,2430.74
